In [ ]:
import pandas as pd
from string import punctuation
import numpy as np
import torch
from nltk.tokenize import word_tokenize
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch import optim
import json
pd.set_option("display.max_column",None)
import nibabel
from torchsample.modules import ModuleTrainer
from sklearn.model_selection import train_test_split

In [ ]:
train_set = pd.read_csv("Train.csv")
test_set = pd.read_csv("Test.csv")
submission = pd.read_csv("SampleSubmission.csv")

In [ ]:
train_set.type.value_counts().values

In [ ]:
all_set = pd.concat([train_set.drop(["type"],axis=1),test_set],axis=0)
all_set.shape

In [ ]:
x_train, x_val_test, y_train, y_val_test = train_test_split(train_set.tweet,train_set.type,test_size=0.3,random_state=42)

In [ ]:
y_train.value

In [ ]:
train_set = x_train
#train_label = y_train.replace({"sexual_violence":0,'Physical_violence':1,'emotional_violence':2,'Harmful_Traditional_practice':3,'economic_violence':4})
train_set.shape

In [ ]:
train_label.unique()

In [ ]:
val_set = x_val_test
#val_label = y_val_test.replace({"sexual_violence":0,'Physical_violence':1,'emotional_violence':2,'Harmful_Traditional_practice':3,'economic_violence':4})
val_set.shape

In [ ]:
from transformers import RobertaTokenizer,MBartTokenizer

tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-en-ro")

def token_counter(text, tokenizer):
    return len(tokenizer.encode(text))

tok_len = all_set.tweet.apply(lambda x : token_counter(x, tokenizer))
max(list(tok_len))

In [ ]:
tokenizer.model_max_length = 169

In [ ]:
import re
def remove_links(text):
    to_remove = ['\r','\n',',',';',':','.']
    
    out = re.sub(r'http\S+', '', text)
    
    for token in to_remove:
        out = out.replace(token, '')
    
    return re.sub(' +', ' ', out.lower()) #Remove duplicate spaces

def tokenize(text, tokenizer):
    return tokenizer.encode(text, padding='max_length')

In [ ]:
#train_set = train_set.apply(remove_links)
train_set = train_set.apply((lambda x : tokenize(x, tokenizer)))

#val_set = val_set.apply(remove_links)
val_set = val_set.apply((lambda x : tokenize(x, tokenizer)))


#test_set.tweet = test_set.tweet.apply(remove_links)
test_set.tweet = test_set.tweet.apply((lambda x : tokenize(x, tokenizer)))

In [ ]:
train_set

In [130]:
train_set

574      [6561, 111, 70, 453, 2750, 29892, 297, 163, 50...
28131    [87, 509, 137802, 4488, 619, 25133, 74, 87, 50...
28560    [4865, 2174, 4, 764, 1902, 29892, 297, 163, 32...
6286     [191096, 22936, 46389, 136, 13416, 4049, 5, 27...
36291    [4263, 87, 36802, 5154, 4, 764, 17688, 538, 12...
                               ...                        
6265     [2646, 1119, 2750, 29892, 297, 163, 159399, 43...
11284    [18852, 759, 40304, 3445, 17669, 83, 103383, 1...
38158    [23972, 5773, 398, 13319, 3714, 163, 566, 959,...
860      [6, 72731, 64, 132191, 647, 63026, 112, 29892,...
15795    [1061, 87, 48948, 1660, 91755, 37838, 5111, 12...
Name: tweet, Length: 27755, dtype: object

array([['sexual_violence'],
       ['sexual_violence'],
       ['sexual_violence'],
       ...,
       ['sexual_violence'],
       ['sexual_violence'],
       ['sexual_violence']], dtype=object)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_label,a)

In [ ]:
train_x, train_y = torch.FloatTensor(train_set), torch.FloatTensor(train_label)
test_x = torch.FloatTensor(test_set.tweet)
valid_x, valid_y = torch.FloatTensor(val_set.reset_index(drop=True)),torch.FloatTensor(val_label.reset_index(drop=True))



train_data = TensorDataset(train_x, train_y)
valid_data = TensorDataset(valid_x, valid_y)
#test_data = TensorDataset(test_x, test_y)


In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset = train_data,
                                          batch_size=batch_size,
                                          shuffle =True)


val_loader = torch.utils.data.DataLoader(dataset=valid_data,
                                          batch_size=batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset=test_x,
                                          batch_size=1,
                                          shuffle = False)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def seed_everything(seed: int):
    import random,os
    import numpy as np 
    import torch 
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministics =True
    torch.backends.cudnn.benchmark = True
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    
seed_everything(42)


import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
class ROBERTAClassifier(torch.nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(ROBERTAClassifier, self).__init__()
        self.e = nn.Embedding(59114,38400)
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.lstm = nn.LSTM(38400, 419, 2,
                    batch_first = True, dropout = 0.3)
        self.l1 = torch.nn.Linear(419,100)
        #self.bn1 = torch.nn.LayerNorm(50)
        #self.l2 = torch.nn.Linear(100, 30)
        self.d1 = torch.nn.Dropout(dropout_rate)
        self.l3 = torch.nn.Linear(100, 5)
        
        
        
        self.d1 = torch.nn.Dropout(dropout_rate)
        
    def forward(self, input_ids):
        x = self.e(input_ids)
        x = self.d1(x[:, -1])
        x, h = self.lstm(x.view(1,1,-1))
        x = self.l1(x)
        #x = self.l1(x.view(1,-1))
        #x = self.bn1(x)
        x = torch.tanh(x)
        x = self.d1(x[:, -1])
        x = self.l3(x)

        
        return x 

In [ ]:
# train mosdel
seed_everything(42)
#model = ROBERTAClassifier()
# Biuld the model
D = 419
model = nn.Sequential(nn.Embedding(59114,300),
                     nn.LSTM(300,64,batch_first=True),
                     nn.Dropout(0.2),
                     nn.Linear(419,250),
                     nn.Linear(250,200),
                     nn.ReLU(),
                     nn.Linear(200,5))
                     #nn.Tanh(),
                     #nn.Linear(50,5))



#loss and optimization
craterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())


n_epochs = 10

#stuffs to store 
train_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)

for it in range(n_epochs):
    train_loss = []
    for inputs,targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        #reshape the input
        #inputs = inputs.view(-1,419)
        # zero the parameter gradients
        optimizer.zero_grad()
        
        #forward pass
        outputs = model(inputs.type(torch.long))
        loss = craterion(outputs,targets.type(torch.LongTensor))
        
        #backward and optimize
        
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
        
        
    train_loss = np.mean(train_loss)
    #for test
    test_loss = []
    
    for inputs,targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        #reshape the input
        #inputs = inputs.view(-1,419)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        #forward pass
        outputs = model(inputs.type(torch.long))
        loss = craterion(outputs,targets.type(torch.LongTensor))
        
        test_loss.append(loss.item())
    test_loss = np.mean(test_loss)
    
    
    #save lossess
    train_losses[it] = train_loss
    test_losses[it] = test_loss
        
        
    
    print(f' {it+1}/{n_epochs}, Loss:{train_loss:4f}, Val_loss:{test_loss}')   
        

In [ ]:
torch.unsqueeze(targets,dim =1).shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses,label="Train_losses")
plt.plot(test_losses, label = "Test_losses")
plt.legend()
plt.show

In [ ]:
n_correct = 0
n_total = 0
for inputs, targets in train_loader:
    #move data to GPU
    inputs, targets =  inputs.to(device),targets.to(device)
    #reshape
    
    inputs = inputs.view(-1,419)
    
    outputs = model(inputs)
    
    _,prediction = torch.max(outputs, 1)
    
    #update counts
    
    n_correct += (prediction == targets).sum().item()
    n_total += targets.shape[0]
    
train_acc = n_correct/n_total

n_correct = 0
n_total = 0
for inputs, targets in val_loader:
    #move data to GPU
    inputs, targets =  inputs.to(device),targets.to(device)
    #reshape
    
    inputs = inputs.view(-1,419)
    
    outputs = model(inputs)
    
    _,prediction = torch.max(outputs, 1)
    
    #update counts
    
    n_correct += (prediction == targets).sum().item()
    n_total += targets.shape[0]
    
test_acc = n_correct/n_total

print(f' Train_Acc:{train_acc},Test_Acc:{test_acc}')

In [ ]:
predict = []
for inputs in test_loader:
    #move data to GPU
    inputs =  inputs.to(device)
    #reshape
    
    inputs = inputs.view(-1,419)
    outputs = model(inputs)
    _,prediction = torch.max(outputs, 1)
    predict.append(prediction.detach().tolist())
    

In [ ]:
df = pd.DataFrame(predict,columns=["type"])
pred = df.replace({0:"sexual_violence",1:'Physical_violence',2:'emotional_violence',3:'Harmful_Traditional_practice',4:'economic_violence'})
idd = submission.Tweet_ID
df = pd.concat([idd,pred],axis=1)

In [ ]:
df.type.nunique()

In [ ]:
df.to_csv('submissionpp.csv',index = False)